In [ ]:
import logging
import os

os.environ["NUMEXPR_MAX_THREADS"] = "8"
import pickle
from importlib import reload

reload(logging)
logger = logging.getLogger()
logging.basicConfig(
    format="%(asctime)s %(message)s", datefmt="%Y-%m-%d %I:%M:%S %p", level=logging.INFO
)

import numpy as np
import parmed as pmd
from openff.toolkit import ForceField as openffForceField
from openff.toolkit import Molecule as openffMolecule
from openff.toolkit import Topology as openffTopology
from openmm import *
from openmm.app import *
from openmm.unit import *
from openmmforcefields.generators import (
    GAFFTemplateGenerator,
    SMIRNOFFTemplateGenerator,
)

compdir = "complex"
datadir = "../../paprika/data/sug-roc"
workdir = "working_data"
for dir in [compdir, datadir, workdir]:
    if not os.path.isdir(dir):
        os.makedirs(dir)

#### Create molecules for the host & guest and assign them types according to GAFF or OpenFF. We'll use Tip3P explicit waters.

In [ ]:
ff = ForceField("amber/tip3p_standard.xml", "amber/tip3p_HFE_multivalent.xml")
guest = openffMolecule.from_file(f"{datadir}/rocuronium.sdf")
guest.generate_unique_atom_names()
if os.path.exists(f"{datadir}/guest_charges.pickle"):
    with open(f"{datadir}/guest_charges.pickle", "rb") as f:
        guest_charges = pickle.load(f)
        guest.partial_charges = guest_charges
else:
    guest.assign_partial_charges(partial_charge_method="am1bcc")
    with open(f"{datadir}/guest_charges.pickle", "wb") as f:
        guest_charges = guest.partial_charges
        pickle.dump(guest_charges, f)

host = openffMolecule.from_file(f"{datadir}/sugammadex.sdf")
host.generate_unique_atom_names()
if os.path.exists(f"{datadir}/host_charges.pickle"):
    with open(f"{datadir}/host_charges.pickle", "rb") as f:
        host_charges = pickle.load(f)
        host.partial_charges = host_charges
else:
    host.assign_partial_charges(partial_charge_method="gasteiger")
    with open(f"{datadir}/host_charges.pickle", "wb") as f:
        host_charges = host.partial_charges
        pickle.dump(host_charges, f)

for a in guest.atoms:
    a.metadata["residue_name"] = "ROC"
for a in host.atoms:
    a.metadata["residue_name"] = "SUG"

gaff = GAFFTemplateGenerator(molecules=[host, guest])
smirnoff = SMIRNOFFTemplateGenerator(molecules=[host, guest], forcefield="openff-2.0.0")
ff.registerTemplateGenerator(
    smirnoff.generator
)  # GAFF or OpenFF generator, user's choice
topology = openffTopology.from_molecules([host, guest])

#### Create an OpenMM system with box dimensions 5x5x10 nanometers using PME and hydrogen mass repartitioning

In [ ]:
pdb = PDBFile(f"{datadir}/sug_roc.pdb")
model = Modeller(topology.to_openmm(ensure_unique_atom_names=True), pdb.positions)
unit_cell_dimensions = Vec3(5.0, 5.0, 10.0) * nanometers
model.topology.setUnitCellDimensions(unit_cell_dimensions)

system = ff.createSystem(
    model.topology,
    nonbondedMethod=PME,
    nonbondedCutoff=0.9 * nanometers,
    rigidWater=True,
    constraints=HBonds,
    hydrogenMass=3.02352 * amu,
)

#### Use pAPRika to align the complex such that the guest is along the z axis. The host ring will lie in the xy plane.

In [ ]:
from paprika.build import align

structure = pmd.openmm.load_topology(
    model.topology,
    system,
    xyz=model.positions,
    condense_atom_types=False,
)

# Specifically, put the line between atoms C14x and C8x on guest residue "ROC" in line with the z axis. Can be changed to other guest residues.
G1 = ":ROC@C14x"
G2 = ":ROC@C8x"

aligned_structure = align.translate_to_origin(structure)
aligned_structure = align.zalign(aligned_structure, G1, G2)

#### Add solvent and dummy atoms

In [ ]:
model = Modeller(
    topology.to_openmm(ensure_unique_atom_names=True), aligned_structure.positions
)
model.topology.setUnitCellDimensions(unit_cell_dimensions)
model.addSolvent(
    ff, boxSize=unit_cell_dimensions, model="tip3p", ionicStrength=5 * millimolar
)  # Here salt can be added or removed
system = ff.createSystem(
    model.topology,
    nonbondedMethod=PME,
    nonbondedCutoff=0.9 * nanometers,
    rigidWater=True,
    constraints=HBonds,
)  # We recreate the OpenMM system because it now has waters

# Give the dummy atoms mass of lead (207.2) and set the nonbonded forces for the dummy atoms to have charge 0 and LJ parameters epsilon=sigma=0.
for force in system.getForces():
    if isinstance(force, openmm.NonbondedForce):
        system.addParticle(mass=207.2)
        system.addParticle(mass=207.2)
        system.addParticle(mass=207.2)
        force.addParticle(0, 0, 0)
        force.addParticle(0, 0, 0)
        force.addParticle(0, 0, 0)

with open(f"{compdir}/solvated_dummy_system.xml", "w") as f:
    f.write(XmlSerializer.serialize(system))

all_positions = model.positions
all_positions.append(Vec3(0.0, 0.0, -12.0) * angstrom)
all_positions.append(Vec3(0.0, 0.0, -15.0) * angstrom)
all_positions.append(Vec3(0.0, 4.33, -17.5) * angstrom)

chain = model.topology.addChain("5")
dummy_res1 = model.topology.addResidue(name="DM1", chain=chain)
dummy_res2 = model.topology.addResidue(name="DM2", chain=chain)
dummy_res3 = model.topology.addResidue(name="DM3", chain=chain)

model.topology.addAtom("DUM", Element.getBySymbol("Pb"), dummy_res1)
model.topology.addAtom("DUM", Element.getBySymbol("Pb"), dummy_res2)
model.topology.addAtom("DUM", Element.getBySymbol("Pb"), dummy_res3)

# Move every atom to the center of the unit cell.
# If we don't do this the complex & solvent will lie on the corner of the unit cell.
all_positions = [
    Vec3(
        p[0].value_in_unit(angstrom)
        + (unit_cell_dimensions[0] / 2).value_in_unit(angstrom),
        p[1].value_in_unit(angstrom)
        + (unit_cell_dimensions[1] / 2).value_in_unit(angstrom),
        p[2].value_in_unit(angstrom)
        + (unit_cell_dimensions[2] / 2).value_in_unit(angstrom),
    )
    for p in all_positions
]

# Move every solute atom 16Å from the center of the solvent box,
# to make room for pulling along z.
for atom in model.topology.atoms():
    if atom.residue.name in ["SUG", "ROC", "DM1", "DM2", "DM3"]:
        all_positions[atom.index] = Vec3(
            all_positions[atom.index][0],
            all_positions[atom.index][1],
            all_positions[atom.index][2] - 16.0,
        )

with open(f"{compdir}/aligned_solvated_dummy_structure.pdb", "w+") as f:
    PDBFile.writeFile(model.topology, all_positions, file=f, keepIds=True)

aligned_solvated_dummy_structure = pmd.openmm.load_topology(
    model.topology, system, xyz=all_positions, condense_atom_types=False
)

In [ ]:
attach_fractions = [f / 100 for f in np.arange(0, 105.0, 5.0)]
initial_distance = 12.0
distance_step = 0.5
pull_distances = np.arange(
    0.0 + initial_distance, 30.0 + initial_distance, distance_step
)
release_fractions = []
windows = [len(attach_fractions), len(pull_distances), len(release_fractions)]
print(f"There are {windows} windows in this APR calculation.")

#### Create static restraints

In [ ]:
from paprika import restraints

G1 = ":ROC@C14x"
G2 = ":ROC@C8x"

H1 = ":SUG@C16x"
H2 = ":SUG@C28x"
H3 = ":SUG@C45x"

D1 = ":DM1@DUM"
D2 = ":DM2@DUM"
D3 = ":DM3@DUM"

static_restraints = []
r = restraints.static_DAT_restraint(
    restraint_mask_list=[D1, H1],
    num_window_list=windows,
    ref_structure=f"{compdir}/aligned_solvated_dummy_structure.pdb",
    force_constant=5.0,
    amber_index=False,  # Because we're using OpenMM
)
static_restraints.append(r)
r = restraints.static_DAT_restraint(
    restraint_mask_list=[H1, H2, H3],
    num_window_list=windows,
    ref_structure=f"{compdir}/aligned_solvated_dummy_structure.pdb",
    force_constant=100.0,
    amber_index=False,
)
static_restraints.append(r)
r = restraints.static_DAT_restraint(
    restraint_mask_list=[D2, D1, H1],
    num_window_list=windows,
    ref_structure=f"{compdir}/aligned_solvated_dummy_structure.pdb",
    force_constant=100.0,
    amber_index=False,
)
static_restraints.append(r)
r = restraints.static_DAT_restraint(
    restraint_mask_list=[D3, D2, D1, H1],
    num_window_list=windows,
    ref_structure=f"{compdir}/aligned_solvated_dummy_structure.pdb",
    force_constant=100.0,
    amber_index=False,
)
static_restraints.append(r)
r = restraints.static_DAT_restraint(
    restraint_mask_list=[D1, H1, H2],
    num_window_list=windows,
    ref_structure=f"{compdir}/aligned_solvated_dummy_structure.pdb",
    force_constant=100.0,
    amber_index=False,
)
static_restraints.append(r)
r = restraints.static_DAT_restraint(
    restraint_mask_list=[D2, D1, H1, H2],
    num_window_list=windows,
    ref_structure=f"{compdir}/aligned_solvated_dummy_structure.pdb",
    force_constant=100.0,
    amber_index=False,
)
static_restraints.append(r)
r = restraints.static_DAT_restraint(
    restraint_mask_list=[D1, H1, H2, H3],
    num_window_list=windows,
    ref_structure=f"{compdir}/aligned_solvated_dummy_structure.pdb",
    force_constant=100.0,
    amber_index=False,
)
static_restraints.append(r)

#### Create dynamic restraints

In [ ]:
G1 = ":ROC@C14x"
G2 = ":ROC@C8x"

guest_restraints = []

r = restraints.DAT_restraint()
r.mask1 = D1
r.mask2 = G1
r.topology = f"{compdir}/aligned_solvated_dummy_structure.pdb"
r.auto_apr = True
r.continuous_apr = True
r.amber_index = False

r.attach["target"] = pull_distances[0]  # Angstroms
r.attach["fraction_list"] = attach_fractions
r.attach["fc_final"] = 2.5  # kcal/mol/Angstroms**2

r.pull["target_final"] = pull_distances[-1]  # Angstroms
r.pull["num_windows"] = windows[1]

r.initialize()
guest_restraints.append(r)

r = restraints.DAT_restraint()
r.mask1 = D2
r.mask2 = D1
r.mask3 = G1
r.topology = f"{compdir}/aligned_solvated_dummy_structure.pdb"
r.auto_apr = True
r.continuous_apr = True
r.amber_index = False

r.attach["target"] = 180.0  # Degrees
r.attach["fraction_list"] = attach_fractions
r.attach["fc_final"] = 80.0  # kcal/mol/radian**2

r.pull["target_final"] = 180.0  # Degrees
r.pull["num_windows"] = windows[1]

r.initialize()
guest_restraints.append(r)

r = restraints.DAT_restraint()
r.mask1 = D1
r.mask2 = G1
r.mask3 = G2
r.topology = f"{compdir}/aligned_solvated_dummy_structure.pdb"
r.auto_apr = True
r.continuous_apr = True
r.amber_index = False

r.attach["target"] = 180.0  # Degrees
r.attach["fraction_list"] = attach_fractions
r.attach["fc_final"] = 80.0  # kcal/mol/radian**2

r.pull["target_final"] = 180.0  # Degrees
r.pull["num_windows"] = windows[1]

r.initialize()
guest_restraints.append(r)

#### We save the restraints in case we need to access them for analysis

In [ ]:
with open(f"{compdir}/static_restraints.pickle", "wb") as f:
    pickle.dump(static_restraints, f)

with open(f"{compdir}/guest_restraints.pickle", "wb") as f:
    pickle.dump(guest_restraints, f)

In [ ]:
from paprika.restraints.utils import create_window_list

window_list = create_window_list(guest_restraints)

#### Since we are using explicit solvent, it's not safe to teleport the guest to different z values to act as the starting point for each pulling window. So to be safe we drag it out with restraints manually.

In [ ]:
import shutil

from paprika.restraints.openmm import apply_dat_restraint, apply_positional_restraints
from paprika.utils import index_from_mask
from tqdm import tqdm

for window in window_list:
    dir = f"{workdir}/{window}"
    if not os.path.isdir(dir):
        os.makedirs(dir)

    if window[0] == "a":
        shutil.copy(
            f"{compdir}/aligned_solvated_dummy_structure.pdb",
            f"{dir}/aligned_solvated_dummy_structure.pdb",
        )

pdb = PDBFile(f"{compdir}/aligned_solvated_dummy_structure.pdb")
old_positions = pdb.positions
step_size = 0.25
assert (
    step_size <= distance_step
), "The step size of the manual pulling must be <= the step size of the automatic pulling step size!"
for offset in tqdm(
    np.arange(min(pull_distances), max(pull_distances) + step_size, step_size)[:]
):
    offset = round(offset, 2)

    with open(f"{compdir}/solvated_dummy_system.xml", "r") as f:
        system = XmlSerializer.deserialize(f.read())

    # Apply dummy static restraints
    apply_positional_restraints(
        f"{compdir}/aligned_solvated_dummy_structure.pdb", system, force_group=15
    )

    # Apply host static restraints
    for restraint in static_restraints:
        apply_dat_restraint(system, restraint, "pull", 1, force_group=10)

    bond_energy_expression = "k_bond * (r - r_0)^2;"
    bond_restraint = CustomBondForce(bond_energy_expression)
    bond_restraint.addPerBondParameter("k_bond")
    bond_restraint.addPerBondParameter("r_0")
    bond_restraint.addBond(
        index_from_mask(f"{compdir}/aligned_solvated_dummy_structure.pdb", D1, False)[
            0
        ],
        index_from_mask(f"{compdir}/aligned_solvated_dummy_structure.pdb", G1, False)[
            0
        ],
        [
            500 * kilocalorie / angstrom**2 / mole,
            (offset) * angstrom,
        ],
    )
    system.addForce(bond_restraint)

    angle_energy_expression = "k_angle * (theta - theta_0)^2;"
    angle_restraint = CustomAngleForce(angle_energy_expression)
    angle_restraint.addPerAngleParameter("k_angle")
    angle_restraint.addPerAngleParameter("theta_0")
    angle_restraint.addAngle(
        index_from_mask(f"{compdir}/aligned_solvated_dummy_structure.pdb", D1, False)[
            0
        ],
        index_from_mask(f"{compdir}/aligned_solvated_dummy_structure.pdb", G1, False)[
            0
        ],
        index_from_mask(f"{compdir}/aligned_solvated_dummy_structure.pdb", G2, False)[
            0
        ],
        [10000 * kilocalorie / mole / radian**2, 180 * degree],
    )
    system.addForce(angle_restraint)

    angle_energy_expression = "k_angle * (theta - theta_0)^2;"
    angle_restraint = CustomAngleForce(angle_energy_expression)
    angle_restraint.addPerAngleParameter("k_angle")
    angle_restraint.addPerAngleParameter("theta_0")
    angle_restraint.addAngle(
        index_from_mask(f"{compdir}/aligned_solvated_dummy_structure.pdb", D2, False)[
            0
        ],
        index_from_mask(f"{compdir}/aligned_solvated_dummy_structure.pdb", D1, False)[
            0
        ],
        index_from_mask(f"{compdir}/aligned_solvated_dummy_structure.pdb", G1, False)[
            0
        ],
        [10000 * kilocalorie / mole / radian**2, 180 * degree],
    )
    system.addForce(angle_restraint)

    integrator = LangevinMiddleIntegrator(
        298.15 * kelvin, 1 / (1000 * femtoseconds), 1 * femtoseconds
    )
    platform = Platform.getPlatformByName("CUDA")
    properties = {"DeviceIndex": "0", "Precision": "mixed"}
    simulation = Simulation(model.topology, system, integrator, platform, properties)
    new_positions = old_positions
    for atom in model.topology.atoms():
        if atom.residue.name == "ROC":
            new_positions[atom.index] = (
                Vec3(
                    old_positions[atom.index][0].value_in_unit(nanometer),
                    old_positions[atom.index][1].value_in_unit(nanometer),
                    (
                        (
                            old_positions[atom.index][2].value_in_unit(angstrom)
                            + step_size
                        )
                        * angstrom
                    ).value_in_unit(nanometer),
                )
                * nanometer
            )
    simulation.context.setPositions(new_positions)
    simulation.minimizeEnergy(
        0.1 * kilocalorie / (mole * nanometer), maxIterations=1000
    )

    new_positions = simulation.context.getState(getPositions=True).getPositions()
    old_positions = new_positions

    if offset in pull_distances:
        PDBFile.writeFile(
            model.topology,
            new_positions,
            open(
                f"{workdir}/p{[float(d) for d in pull_distances].index(float(offset)):03}/aligned_solvated_dummy_structure.pdb",
                "w",
            ),
        )

        PDBFile.writeFile(
            model.topology,
            new_positions,
            open(
                f"{compdir}/all_starting_positions.pdb",
                "a",
            ),
        )

#### Now we can run APR like usual

In [ ]:
from paprika.restraints.utils import parse_window

logging.info(f"Creating xml files...")
for window in tqdm(window_list):
    window_number, phase = parse_window(window)
    # logging.info(f"Creating XML for in window {window}")

    with open(f"{compdir}/solvated_dummy_system.xml", "r") as f:
        system = xmlser.deserialize(f.read())

    # Apply dummy static restraints
    apply_positional_restraints(
        f"{workdir}/{window}/aligned_solvated_dummy_structure.pdb",
        system,
        force_group=15,
    )

    # Apply host static restraints
    for restraint in static_restraints:
        apply_dat_restraint(system, restraint, phase, window_number, force_group=10)

    # Apply guest restraints
    for restraint in guest_restraints:
        apply_dat_restraint(system, restraint, phase, window_number, force_group=11)

    # Save OpenMM system to XML file
    with open(f"{workdir}/{window}/system.xml", "w") as file:
        file.write(xmlser.serialize(system))

In [ ]:
# Run minimization

logging.info(f"Running minimization...")
for window in tqdm(window_list):
    # logging.info(f"Running minimization in window {window}...")

    # Load XML and input coordinates
    with open(f"{workdir}/{window}/system.xml", "r") as f:
        system = xmlser.deserialize(f.read())
    pdb = PDBFile(f"{workdir}/{window}/aligned_solvated_dummy_structure.pdb")

    integrator = LangevinMiddleIntegrator(
        298.15 * kelvin, 1 / (200 * femtoseconds), 1.0 * femtoseconds
    )
    platform = Platform.getPlatformByName("CUDA")
    properties = {"DeviceIndex": "0", "Precision": "mixed"}
    simulation = Simulation(model.topology, system, integrator, platform, properties)
    simulation.context.setPositions(pdb.positions)
    simulation.minimizeEnergy(
        tolerance=0.0001 * kilojoules_per_mole / nanometer, maxIterations=1000
    )

    positions = simulation.context.getState(getPositions=True).getPositions()

    with open(f"{workdir}/{window}/minimized.pdb", "w") as f:
        app.PDBFile.writeFile(model.topology, positions, f, keepIds=True)

In [ ]:
# Run heating. This is just to be safe.

logging.info(f"Running heating...")
for window in tqdm(window_list):
    # logging.info(f"Running heating in window {window}...")

    # Load XML and input coordinates
    with open(f"{workdir}/{window}/system.xml", "r") as f:
        system = xmlser.deserialize(f.read())
    pdb = PDBFile(f"{workdir}/{window}/minimized.pdb")

    integrator = LangevinMiddleIntegrator(
        298.15 * kelvin, 1 / (1000 * femtoseconds), 1.0 * femtoseconds
    )
    platform = Platform.getPlatformByName("CUDA")
    properties = {"DeviceIndex": "0", "Precision": "mixed"}
    simulation = Simulation(model.topology, system, integrator, platform, properties)
    simulation.context.setPositions(pdb.positions)

    dcd_reporter = DCDReporter(f"{workdir}/{window}/heating.dcd", 500)
    state_reporter = StateDataReporter(
        f"{workdir}/{window}/heating.log",
        500,
        step=True,
        kineticEnergy=True,
        potentialEnergy=True,
        totalEnergy=True,
        temperature=True,
        volume=True,
        speed=True,
    )

    simulation.reporters.append(dcd_reporter)
    simulation.reporters.append(state_reporter)

    for temp in np.arange(200.0, 298.15, 10.0):
        integrator.setTemperature(temp * kelvin)
        simulation.step(2000)

    positions = simulation.context.getState(getPositions=True).getPositions()

    with open(f"{workdir}/{window}/heated.pdb", "w") as f:
        app.PDBFile.writeFile(model.topology, positions, f, keepIds=True)

In [ ]:
# Run production

logging.info(f"Running production...")
for window in tqdm(window_list):
    # logging.info(f"Running production in window {window}...")

    # Load XML and input coordinates
    with open(f"{workdir}/{window}/system.xml", "r") as f:
        system = xmlser.deserialize(f.read())
    pdb = PDBFile(f"{workdir}/{window}/minimized.pdb")

    integrator = LangevinMiddleIntegrator(
        298.15 * kelvin, 1 / (1000 * femtoseconds), 1 * femtoseconds
    )
    platform = Platform.getPlatformByName("CUDA")
    properties = {
        "DeviceIndex": "0",
        "Precision": "mixed",
    }  # If you have multiple GPUs, change "DeviceIndex": "0,1,2,3" etc.
    simulation = Simulation(model.topology, system, integrator, platform, properties)
    simulation.context.setPositions(pdb.positions)

    dcd_reporter = DCDReporter(f"{workdir}/{window}/production.dcd", 500)
    state_reporter = StateDataReporter(
        f"{workdir}/{window}/production.log",
        500,
        step=True,
        kineticEnergy=True,
        potentialEnergy=True,
        totalEnergy=True,
        temperature=True,
        volume=True,
        speed=True,
    )

    simulation.reporters.append(dcd_reporter)
    simulation.reporters.append(state_reporter)

    simulation.step(
        100000
    )  # 100 ps -- this seems to be enough to get energy convergence in each window

    positions = simulation.context.getState(getPositions=True).getPositions()

    with open(f"{workdir}/{window}/production.pdb", "w") as f:
        app.PDBFile.writeFile(model.topology, positions, f, keepIds=True)

#### Perform analysis

In [ ]:
import paprika.analysis as analysis

free_energy = analysis.fe_calc()
free_energy.topology = "minimized.pdb"
free_energy.trajectory = "production.dcd"
free_energy.path = workdir
free_energy.restraint_list = guest_restraints
free_energy.collect_data()
free_energy.methods = ["ti-block"]
free_energy.ti_matrix = "full"
free_energy.boot_cycles = 1000
free_energy.compute_free_energy()

In [ ]:
free_energy.compute_ref_state_work(
    [guest_restraints[0], guest_restraints[1], None, None, guest_restraints[2], None]
)

binding_affinity = -1 * (
    free_energy.results["attach"]["ti-block"]["fe"]
    + free_energy.results["pull"]["ti-block"]["fe"]
    + free_energy.results["ref_state_work"]
)

sem = np.sqrt(
    free_energy.results["attach"]["ti-block"]["sem"] ** 2
    + free_energy.results["pull"]["ti-block"]["sem"] ** 2
)

result_string = f"Binding affinity: {binding_affinity:0.4f} ± {sem:0.4f}"
print(result_string)
with open("result.txt", "w") as f:
    f.write(result_string)

The experimental binding free energy is -8.27 kcal/mol<sup>1</sup>. To get a more accurate calculation we can increase the number of attach fractions, increase the pull distance somewhat, decrease the step size between pull distances, to name a few options. Also realize that here we are only calculating the binding of rocuronium in one orientation to one face of sugammadex (primary).

(1) https://doi.org/10.1016/j.molliq.2018.06.033